[View in Colaboratory](https://colab.research.google.com/github/snowdj/examples/blob/master/README.ipynb)

Run these codes first in order to install the necessary libraries and perform authorization.

https://www.kdnuggets.com/2018/02/google-colab-free-gpu-tutorial-tensorflow-keras-pytorch.html

In [7]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [0]:
# Click the link, copy verification code and paste it to text box.

# After completion of the authorization process, mount your Google Drive:

!mkdir -p drive
!google-drive-ocamlfuse drive

In [9]:
!ls

ComputationalEconomics	macroeconomics	     README.md
custom.css		math		     resourceEconomics
Data			Media_Example.ipynb  style.css
drive			microeconomics	     __Template__.ipynb
econometrics		other


In [0]:
!pip install -q keras

In [1]:
!git clone https://github.com/jonduan/Economics_note.git

Cloning into 'Economics_note'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2451 (delta 11), reused 18 (delta 7), pack-reused 2427
Receiving objects: 100% (2451/2451), 110.83 MiB | 29.51 MiB/s, done.
Resolving deltas: 100% (1361/1361), done.
Checking out files: 100% (1034/1034), done.


In [2]:
!ls


Economics_note	sample_data


In [0]:
!cd Economics_note/

In [4]:
!ls


Economics_note	sample_data


In [0]:
import os
os.chdir('Economics_note')

In [6]:
!ls

ComputationalEconomics	math		     resourceEconomics
custom.css		Media_Example.ipynb  style.css
Data			microeconomics	     __Template__.ipynb
econometrics		other
macroeconomics		README.md


https://chrome.google.com/webstore/detail/curlwget/jmocjfidanebdlinpbcdkcmgdifblncg/related?hl=en


Actually I found a solution to Kaggle -> Colab direct transfer.

I used the CurlWget extension Jeremy describes here [https://youtu.be/9C06ZPF8Uuc?t=744 30] and then invoke !wget ... from Colab to download into my Google Drive. Extremely fast transfer all within Google Cloud.


https://theailearner.com/2018/03/10/free-gpu-for-fast-ai-on-google-colab/

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)